# Введение
Цель данного ноутбука заключается в создании модели, способной отличать кликбейтные тексты от обычных.

# Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import warnings
warnings.filterwarnings('ignore')

2024-04-09 13:36:47.792766: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Загружаем данные
Мы рассматриваем 3 датасета *(один с kaggle, другие два сгенерированы с помощью YandexGPT)*

## Датасеты сгенерированные с помощью YandexGPT
Они были сгенерированы в разное время, поэтому их структура немного отличается друг от друга.

In [2]:
FILE_1 = 'files/yandexgpt_generated_1.csv'
FILE_2 = 'files/yandexgpt_generated_2.csv'

temp1 = pd.read_csv(FILE_1, index_col='index')
temp2 = pd.read_csv(FILE_2, index_col='index')

temp1 = temp1[['title', 'clickbait_title']]
temp2 = temp2[['title', 'clickbait']]

temp2.rename(columns={'clickbait':'clickbait_title'}, inplace=True)

gpt_generated = pd.concat([temp1, temp2])

In [3]:
gpt_generated.sample(10)

,title,clickbait_title
index,,
1902,Россияне массово не захотели переезжать в четы...,"«Четыре российских города, в которые лучше не ..."
2886,Иностранные самолеты останутся в России вопрек...,"К сожалению, я не могу ничего сказать об этом...."
1804,Посольство России оценило вручение Пулитцеровс...,"К сожалению, я не могу ничего сказать об этом...."
4625,Роналду побил мировой рекорд по голам за сборную,**Криштиану Роналду вошёл в историю! Как Порту...
617,Россияне выбрали лучшие фильмы и сериалы года,Вот один из возможных вариантов заголовка:\r\n...
3096,Блогерша раскрыла секрет идеального завтрака и...,«Яичный сэндвич от @Ivycher — новый кулинарный...
4247,Названы основные способы мошенничества с жилье...,«Не открывайте дверь! Названы главные схемы мо...
3745,Совладелец WhatsApp заполучил особняк Синди Кр...,«Синди Кроуфорд рассталась с роскошным особняк...
3789,В России раскрыли масштабную аферу с IKEA посл...,«Шокирующая афера предпринимательницы из Ворон...


#### Объединим 2 колонки в одну

In [4]:
not_clickbait = pd.DataFrame({'title': gpt_generated['title']})
not_clickbait['is_clickbait'] = 0

clickbait = pd.DataFrame({'title': gpt_generated['clickbait_title']})
clickbait['is_clickbait'] = 1

df = pd.concat([clickbait, not_clickbait])

In [5]:
df.sample(5)

,title,is_clickbait
index,,
4105,Названы самые популярные зарубежные направлени...,0
239,**Загадочная эпидемия: сотни людей оказались з...,1
2179,"К сожалению, я не могу ничего сказать об этом....",1
237,"«Мечтаете переехать в Испанию? Узнайте, жители...",1
513,«Краснодар» теряет ключевого игрока: что произ...,1


## Датасет с Kaggle



In [6]:
FILE_3 = 'files/kaggle.csv'
kaggle = pd.read_csv(FILE_3, engine='python', sep=';')
kaggle.sample(5)

,titles,target
2218,Сотрудничество с Россией в космической отрасли...,0
1401,Глаз не оторвать: Толкалина показала откровенн...,1
2337,На Землю вернулась капсула с грунтом с астерои...,0
3087,СМИ: израильские банки начали сегрегировать сч...,0
2901,Израиль вынудит ХАМАС обменять пленных после в...,0


In [7]:
kaggle.rename(columns={'titles' : 'title', 'target' : 'is_clickbait'}, inplace=True)
df = pd.concat([kaggle, df])

In [8]:
df.sample(10)

,title,is_clickbait
4152,Дачников призвали срочно раскидать кроличий на...,0
3153,Стало известно о резко подешевевших для россия...,0
988,**Алоэ — бесполезное и опасное для здоровья ра...,1
935,Признавшуюся в ненависти к Польше гимнастку на...,0
1791,Вяльбе рассказала о взятом Большуновым терапев...,0
440,Влияние нового сбора для туристов на стоимость...,0
3126,СМИ: послы стран ЕС планируют обсудить 12-й па...,0
2924,Девушка написала одно сообщение от лица сестры...,0
489,«Попрошайничество в отеле: как система „всё вк...,1
1380,«Жалкий вид»: фанаты прошлись по уехавшей Чулп...,1


# Очистим данные

In [9]:
# Удалим дубликаты и пропуски
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Удалим заголовки, на которые YandexGPT не дал ответы
no_answer = 'К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?'
df = df[~df['title'].str.startswith(no_answer)]

# Удалим ненужную информацию из заголовков
def clean(text):
    text = text.strip()
    # Фраза предлагающая один из вариантов кликбейта
    if text.startswith('Вот один из'):
        parts = [part.strip() for part in text.split(':')[1:]]
        text = ' '.join(parts)
    # Фраза предлагающая несколько вариантов кликбейта
    if text.startswith('Вот несколько'):
        text = text.split(':')[1].strip()
    # Ненужные знаки при генерации текста
    useless = ['«', '»', '**', '*']
    for to_change in useless:
        text = text.replace(to_change, '')
    return text

df['title'] = df['title'].apply(clean)

In [10]:
# Баланс классов после очистки
df['is_clickbait'].value_counts()

is_clickbait
0    7529
1    6395
Name: count, dtype: int64

In [11]:
# Посмотрим 10 прозвольных заголовков после очистки
for x in df.sample(10)['title']:
    print(x)

Пенсии повысят еще на 20%. Пенсионерам объявили о приятном сюрпризе
Девушка взглянула на мусорное ведро бойфренда и заподозрила его в измене
Нурмагомедов уходит из MMA: что ждёт непобеждённого чемпиона в боксе?
Браво в Новый год: хиты легендарной группы, танцы до утра и фуршет в подарок!
В России рассказали о нехватке мест для детей из-за человейников
Легендарный советский легкоатлет ушёл из жизни: что стало причиной трагедии?
Петр Ян объяснил запрещенный удар в бое за титул UFC
К журналистке пришли с обыском и увезли ее в неизвестном направлении
Овечкин пропустит чемпионат мира по хоккею
В Ахмате прокомментировали слова Кадырова о Нурмагомедове


# Разбиваем данные

In [13]:
text = df['title'].values
labels = df['is_clickbait'].values
text_train, text_test, y_train, y_test = train_test_split(text, labels, test_size=0.15)
text_train, train_val, y_train, y_val = train_test_split(text_train, y_train, test_size=0.1)

print(f'Train: X~{text_train.shape[0]}, y~{y_train.shape[0]}')
print(f'Test: X~{text_test.shape[0]}, y~{y_test.shape[0]}')
print(f'Validation: X~{train_val.shape[0]}, y~{y_val.shape[0]}')

Train: X~10651, y~10651
Test: X~2089, y~2089
Validation: X~1184, y~1184


# Токенизация

In [14]:
vocab_size = 5000
maxlen = 100
embedding_size = 32

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(text)

X_train = tokenizer.texts_to_sequences(text_train)
x_test = tokenizer.texts_to_sequences(text_test)
x_val = tokenizer.texts_to_sequences(train_val)

X_train = pad_sequences(X_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
x_val = pad_sequences(x_val, maxlen=maxlen)

# Обучение модели

In [15]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=maxlen))
model.add(LSTM(32, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

2024-04-09 13:37:31.515164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100, 32)           8320      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 168,353
Trainable params: 168,353
Non-trainable params: 0
__________________________________________________

In [16]:
callback = [
    EarlyStopping(
        monitor='val_accuracy',
        min_delta=1e-4,
        patience=3,
        verbose=1
    )]

In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=64, validation_data=(x_val, y_val), epochs=10, callbacks=callback)

Epoch 1/10
167/167 [==============================] - 9s 43ms/step - loss: 0.6385 - accuracy: 0.6777 - val_loss: 0.4671 - val_accuracy: 0.8970
Epoch 2/10
167/167 [==============================] - 7s 43ms/step - loss: 0.3036 - accuracy: 0.9154 - val_loss: 0.2119 - val_accuracy: 0.9350
Epoch 3/10
167/167 [==============================] - 7s 40ms/step - loss: 0.1650 - accuracy: 0.9516 - val_loss: 0.1848 - val_accuracy: 0.9367
Epoch 4/10
167/167 [==============================] - 6s 38ms/step - loss: 0.1155 - accuracy: 0.9684 - val_loss: 0.1850 - val_accuracy: 0.9358
Epoch 5/10
167/167 [==============================] - 6s 37ms/step - loss: 0.0876 - accuracy: 0.9771 - val_loss: 0.2092 - val_accuracy: 0.9248
Epoch 6/10
167/167 [==============================] - 8s 45ms/step - loss: 0.0721 - accuracy: 0.9835 - val_loss: 0.2198 - val_accuracy: 0.9299
Epoch 6: early stopping


# Проверка модели

In [19]:
predictions_probability = model.predict(x_test)
predictions = [round(x[0]) for x in predictions_probability] # округляем до 0 или 1
print(classification_report(y_test, predictions))

66/66 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      1132
           1       0.91      0.91      0.91       957

    accuracy                           0.92      2089
   macro avg       0.92      0.92      0.92      2089
weighted avg       0.92      0.92      0.92      2089



# Сохраняем модель

In [20]:
model.save('clicbait_classifier.keras')